In [62]:
import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Charger les données
data = pd.read_csv('/Users/cosaph/Desktop/fc servette/data_model.csv')

player = 'J.Simo'
data_player = data[data['PLAYER_NAME'] == player]

# Fonction pour générer des sessions futures (à adapter à tes données)
def generate_future_sessions(current_sessions, steps):
    last_session = current_sessions.iloc[-1]
    future_sessions = []
    for i in range(1, steps + 1):
        new_session = f"{last_session}_{i}"
        future_sessions.append(new_session)
    return pd.Series(future_sessions)

features = ['Distance m/min', 'Nombre de sprints', 'Accélération max', 'Nb acc [4/5] ms²', 'Vmax']
data = data_player[features]
df = pd.DataFrame(data)

# Calcul du z-score pour chaque feature
df = (df - df.mean()) / df.std()
df_normalized = (df - df.min()) / (df.max() - df.min()) # entre 0 et 1

# Calcul de la somme des z-scores normalisés
df_normalized['sum_z_scores'] = df_normalized.sum(axis=1)
#print(df_normalized)

# Lissage exponentiel sur la somme des z-scores normalisés
# Single Exponential Smoothing
model = ExponentialSmoothing(df_normalized['sum_z_scores'], trend='add')
model_fit = model.fit()
smoothed_data = model_fit.fittedvalues

# Prédictions futures
future_steps = 10
future_predictions = model_fit.forecast(steps=future_steps)
future_sessions = generate_future_sessions(data_player['SESSION_ID'], future_steps)

# Calcul de la variation en pourcentage relatif de la moyenne glissante
initial_value = smoothed_data.iloc[0]
final_value = smoothed_data.iloc[-1]
print(f"Valeur initiale: {initial_value:.2f}")
print(f"Valeur finale: {final_value:.2f}")

percentage_change = ((final_value - initial_value) / initial_value) * 100

print(f"Variation en pourcentage du développement du joueur: {percentage_change:.2f}%")

# calculer la variation en pourcentage pour la prédiction future
future_percentage_change = ((future_predictions.iloc[-1] - final_value) / final_value) * 100
print(f"Variation en pourcentage de la prédiction future: {future_percentage_change:.2f}%")



Valeur initiale: 1.82
Valeur finale: 2.40
Variation en pourcentage du développement du joueur: 32.46%
Variation en pourcentage de la prédiction future: 7.21%


/Users/cosaph/Desktop/fc servette/mon_env/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/Users/cosaph/Desktop/fc servette/mon_env/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

/Users/cosaph/Desktop/fc servette/mon_env/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.



In [63]:
# Tracé des z-scores en fonction de SESSION_ID

# session id renaming in "match 1", "match 2", etc in len(math_id)
data_player['SESSION_ID'] = ['match ' + str(i) for i in range(1, len(data_player['SESSION_ID']) + 1)]


fig = make_subplots(rows=1, cols=1)

fig.add_trace(go.Scatter(x=data_player['SESSION_ID'], y=df_normalized['sum_z_scores'], mode='lines', name='Somme des z-scores normalisés'), row=1, col=1)
fig.add_trace(go.Scatter(x=data_player['SESSION_ID'], y=smoothed_data, mode='lines', name='Lissage exponentiel'), row=1, col=1)
fig.add_trace(go.Scatter(x=future_sessions, y=future_predictions, mode='lines', name='Prédictions futures'), row=1, col=1)


fig.update_xaxes(tickvals=data_player['SESSION_ID'][::5], ticktext=data_player['SESSION_ID'][::5], row=1, col=1)

fig.update_layout(title_text=f'Somme des z-scores normalisés et lissage exponentiel (Variation: {percentage_change:.2f}%)')
fig.show()